In [9]:
import json
import os

from tqdm import tqdm
import pandas as pd
import jams
import librosa
import madmom
import crema
import pumpp

In [4]:
def ann_to_label(ann):
    intervals, labels = ann.to_interval_values()
    dur_tracker = dict()

    for interval, label in zip(intervals, labels):
        dur = interval[1] - interval[0]
        if label not in dur_tracker:
            dur_tracker[label] = dur
        else:
            dur_tracker[label] += dur

    # return the key with the max value as the gloabl key_mode
    return max(dur_tracker, key=dur_tracker.get)


class MadmomKeyAnalyzer(object):
    def __init__(self):
        self.key_processor = madmom.features.key.CNNKeyRecognitionProcessor()

    def to_label(self, filename):
        return madmom.features.key.key_prediction_to_label(self.key_processor(filename))
    
    def to_ann(self, filename):
        label = self.to_label(filename)
        label = ':'.join(label.split(' '))
        file_dur = librosa.get_duration(filename=filename)
        ann = jams.Annotation(namespace='key_mode', time=0, duration=file_dur)
        ann.append(time=0, duration=file_dur, value=label)
        return ann
    
    
class CremaKeyAnalyzer(object):
    def __init__(self):
        self.model = crema.models.key.KeyModel()
    
    def to_ann(self, filename):
        return self.model.predict(filename=filename)
    
    def to_label(self, filename):
        ann = self.to_ann(filename)
        label = ann_to_label(ann)
        return ' '.join(label.split(':'))

In [5]:
mka = MadmomKeyAnalyzer()
cka = CremaKeyAnalyzer()

In [6]:
f_name = librosa.util.example_audio_file()

m_label = mka.to_label(f_name)
c_label = cka.to_label(f_name)

In [7]:
m_label, c_label

('E major', 'E major')

## Next steps
- convert ground truth to label and to jams
- convert madmom label to jams and compare there
- *optional* convert crema jams to label and compare there

In [8]:
DATA_HOME = "/Users/tom/Music/GuitarSet/"

with open('val_track_ids.json', 'r') as fp:
    val_track_ids = json.load(fp)

with open('guitarset_index.json', 'r') as fp:
    guitarset_idx = json.load(fp)

In [35]:
label_true = []
label_madmom = []
label_crema = []

for val_id in tqdm(val_track_ids):
    val_id = val_id[:-4]
    jam_path = os.path.join(DATA_HOME, guitarset_idx[val_id]['jams'][0])
    mic_path = os.path.join(DATA_HOME, guitarset_idx[val_id]['audio_mic'][0])
    
    ann_true = jams.load(jam_path).search(namespace='key_mode')[0]
    l_true = ann_to_label(ann_true)
    label_true.append(' '.join(l_true.split(':')))
    label_madmom.append(mka.to_label(mic_path))
    label_crema.append(cka.to_label(mic_path))

result = pd.DataFrame(data={'truth':label_true, 'madmom':label_madmom, 'crema':label_crema}, 
                      index=val_track_ids)

100%|██████████| 72/72 [06:33<00:00,  5.70s/it]


In [36]:
result

,truth,madmom,crema
05_Funk3-112-C#_comp_mic,C# major,Ab major,C# major
01_Rock1-130-A_comp_mic,A major,A major,A major
04_BN3-154-E_solo_mic,E major,E major,E major
02_SS2-107-Ab_solo_mic,Ab minor,B major,G# minor
00_Funk2-119-G_solo_mic,G minor,F major,A# major
...,...,...,...
00_BN1-129-Eb_comp_mic,Eb major,Bb major,D# major
03_Rock1-90-C#_solo_mic,C# major,E major,C# major
02_SS3-84-Bb_comp_mic,Bb major,Eb major,A# major
02_Rock3-148-C_solo_mic,C major,C major,C major


In [37]:
import mir_eval

In [38]:
t_score = 0
m_score = 0
c_score = 0

for track_id, row in result.iterrows():
    t_score += mir_eval.key.evaluate(reference_key=row.truth, estimated_key=row.truth)['Weighted Score']
    m_score += mir_eval.key.evaluate(reference_key=row.truth, estimated_key=row.madmom)['Weighted Score']
    c_score += mir_eval.key.evaluate(reference_key=row.truth, estimated_key=row.crema)['Weighted Score']
    
print('weighted mirex scores:')
print('\t madmom : {}'.format(m_score / t_score))
print('\t crema  : {}'.format(c_score / t_score))

weighted mirex scores:
	 madmom : 0.5527777777777777
	 crema  : 0.9152777777777779
